In [1]:
import nltk
nltk.download('gutenberg')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import gutenberg
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
from collections import defaultdict
import re
import itertools
import json
import numpy as np
from scipy.sparse import lil_matrix

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\aksha\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aksha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aksha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def clean_corpus(words):
   stop_words = set(stopwords.words('english'))
   pattern = re.compile("^[a-zA-Z]+$")
   readycorpus=[]
   for ch in words:
       ch = ch.lower()
       if pattern.match(ch) and ch not in stop_words and ch != '.' and ch !=" ":
            readycorpus.append(ch)
   return readycorpus

In [3]:
file1 = open("blah.txt","r",encoding="utf8")
wikicorpous=file1.read()
file1.close()

In [4]:
corpus = gutenberg.sents()
gutenbergsentlist=list(corpus)
guten_window_corpus=[]
fullgutencorpus=[]
for sent in tqdm(gutenbergsentlist):
    cleaned_raw= clean_corpus(sent)
    if len(cleaned_raw)>=3:
        fullgutencorpus+=cleaned_raw
        guten_window_corpus.append(cleaned_raw)

  0%|          | 0/98552 [00:00<?, ?it/s]

In [5]:
wikilist=wikicorpous.split(".")
wikisentlist=[]
for sent in wikilist:
     wikisentlist.append(sent.split(" "))

In [6]:
wiki_window_corpus=[]
fullwikicorpus=[]
for sent in tqdm(wikisentlist):
    cleaned_raw= clean_corpus(sent)
    if len(cleaned_raw)>=3:
        fullwikicorpus+=cleaned_raw
        wiki_window_corpus.append(cleaned_raw)

  0%|          | 0/16969 [00:00<?, ?it/s]

In [7]:
fullcorpus=fullgutencorpus+fullwikicorpus
windowfullcorpus=guten_window_corpus+wiki_window_corpus

In [8]:
vocabulary=list(sorted(set(fullcorpus)))
print(f"vocabulary={len(vocabulary)}")
vocabulary[0:5]

vocabulary=49245


['aadmi', 'aam', 'aaron', 'aaronites', 'aarthik']

In [9]:
index = 0
wordxInd = {}
Indxword = {}
for word in vocabulary:
    if word not in wordxInd: 
        wordxInd[word] = index
        Indxword[index] = word
        index += 1

In [10]:
def get_windows(corpus, C):
    words=corpus
    i=0
    while i < len(words):
        center_word = words[i]
        context_words = words[(i - C):i] + words[(i+1):(i+C+1)]
        yield context_words, center_word
        i += 1

In [11]:
cop=[]
for li in windowfullcorpus:
    cop+=li

In [12]:
leng=0
for context,center in get_windows(cop,5):
    leng+=1
print(f"batch training samples : {leng}")

batch training samples : 1209738


In [13]:
def getxy_skipgram(V,leng,window_corpus,wordxInd,windowsize):
    x_dumy=lil_matrix((V,leng))
    y_dumy=lil_matrix((V,leng))
    for i,words in tqdm(enumerate(get_windows(window_corpus,windowsize))):
        x_dumy[wordxInd[words[1]],i]=1
        for word in words[0]:
            y_dumy[wordxInd[word],i]=1
    return x_dumy,y_dumy

In [14]:
x_dumy,y_dumy=getxy_skipgram(len(vocabulary),leng,cop,wordxInd,5)

0it [00:00, ?it/s]

In [15]:
import scipy.sparse as sp
x_dumy=x_dumy.tocsr(copy=False)
sp.save_npz('x_skip_win5.npz', x_dumy)

In [16]:
import scipy.sparse as sp
y_dumy=y_dumy.tocsr(copy=False)
sp.save_npz('y_skip_win5.npz', y_dumy)

In [17]:
def getxy_cbow(V,leng,window_corpus,wordxInd,windowsize):
  x_dumy=lil_matrix((V,leng))
  y_dumy=lil_matrix((V,leng))
  for i,words in enumerate(get_windows(window_corpus, windowsize)):
    y_dumy[wordxInd[words[1]],i]=1
    for word in words[0]:
        x_dumy[wordxInd[word],i]=1
  return x_dumy,y_dumy

In [18]:
x_dumy1,y_dumy1=getxy_cbow(len(vocabulary),leng,cop,wordxInd,5)

In [19]:
import scipy.sparse as sp
x_dumy1=x_dumy1.tocsr(copy=False)
sp.save_npz('x_cbow_win5.npz', x_dumy1)

In [20]:
import scipy.sparse as sp
y_dumy1=y_dumy1.tocsr(copy=False)
sp.save_npz('x_cbow_win5.npz', y_dumy1)

In [65]:
W0=np.random.uniform(low=-0.8, high=0.8, size=(V,dims))
W1=np.random.uniform(low=-0.8, high=0.8, size=(dims,V))

In [79]:
def softmax(x):
    exp_z = np.exp(x)
    sum_exp_z = np.sum(exp_z,axis=0,keepdims=True)
    softmax_probs = exp_z / sum_exp_z
    return softmax_probs

In [80]:
def forward(W0,W1,x):
    h=np.dot(W0.T,x)
    v=np.dot(W1.T,h)
    return h,softmax(v)

In [81]:
def backprop(W0,W1,x,y_true,alpha):
    h,y_pred=forward(W0,W1,x)
    y_pred=np.maximum(1e-8,y_pred)
    loss = - np.sum(y_true*np.log(y_pred))/y_true.shape[1]
    l1 = np.dot(W1 ,(y_pred - y_true))
    dW1 = np.dot(h,(y_pred - y_true).T)
    dW0 = np.dot(x,l1.T)
    W1 -= alpha * dW1
    W0 -= alpha * dW0
    return W0,W1,loss

In [82]:
def minibatch(W0,W1,x,y,leng,epochs):
    W0x,W1x=W0,W1
    e=0
    while e<=epochs:
        info=""
        batchsize=100
        i=0
        while batchsize+i <= leng:    
            W0x,W1x,loss=backprop(W0x,W1x,np.asarray(x[:,i:i+batchsize].toarray()),np.asarray(y[:,i:i+batchsize].toarray()),0.03)
            if (i+batchsize)% 50000==0:
                info+=f"loss at epoch {e} with batch {i} - {i+batchsize} is {loss}\n"
                print(f"loss at epoch {e} with batch {i} - {i+batchsize} is {loss}")
          
            if i+batchsize == leng:
                info+=f"returned to batch of epoch {e} with loss at {i} - {i+batchsize} is {loss}\n"
                print(f"returned to batch of epoch {e} with loss at {i} - {i+batchsize} is {loss}")

                if not np.isnan(W0x[1,2]):
                    print(f"content w0,w1 saved for epoch {e} at {i+batchsize}")
                    np.save(f'W0epoch{e}.npy',W0x)
                    np.save(f'W1epoch{e}.npy',W1x)
                    with open('loss.txt', 'a') as file:
                          file.write(info)

            if i+batchsize == 659400:
              batchsize=(leng-(batchsize+i))
          
            i+=100
      
        e+=1

In [83]:
leng=x.shape[1]

In [ ]:
minibatch(W0,W1,x,y,leng,25)